In [ ]:
! pip install pyLDAvis gensim spacy

In [1]:
pip install pyldavis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 48.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 31.3 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.26.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.1 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.2.1 which is incompatible.


**#import library**

In [2]:
import pandas as pd
import re
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import Phrases
import spacy
from collections import defaultdict
from gensim import corpora, models
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

**#include and read the json file**

In [3]:
file_path = "https://raw.githubusercontent.com/pujan08/news_modeling/main/newsgroups.json"
df = pd.read_json(file_path)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**#Preprocess the data**

**#Remove email address**

In [4]:
# Define a function to remove email addresses from text
def remove_emails(text):
    # Define the regex pattern for detecting email addresses
    pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    # Replace email addresses with an empty string
    return re.sub(pattern, '', text)
# Apply remove_emails function to each text in the DataFrame
df['content'] = df['content'].apply(remove_emails)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**#Newline Removal**

In [5]:
def remove_newlines(text):
    # Define the regex pattern for detecting newline characters
    pattern = r'\n'
    # Replace newline characters with a space
    return re.sub(pattern, ' ', text)
# Apply remove_newlines function to each text in the DataFrame
df['content'] = df['content'].apply(remove_newlines)
# Display the cleaned texts
for index, row in df.head().iterrows():
    print(f"Text {index}: {row['content']}\n")

Text 0: From:  (where's my thing) Subject: WHAT car is this!? Nntp-Posting-Host: rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: 15   I was wondering if anyone out there could enlighten me on this car I saw the other day. It was a 2-door sports car, looked to be from the late 60s/ early 70s. It was called a Bricklin. The doors were really small. In addition, the front bumper was separate from the rest of the body. This is  all I know. If anyone can tellme a model name, engine specs, years of production, where this car is made, history, or whatever info you have on this funky looking car, please e-mail.  Thanks, - IL    ---- brought to you by your neighborhood Lerxst ----     

Text 1: From:  (Guy Kuo) Subject: SI Clock Poll - Final Call Summary: Final call for SI clock reports Keywords: SI,acceleration,clock,upgrade Article-I.D.: shelley.1qvfo9INNc3s Organization: University of Washington Lines: 11 NNTP-Posting-Host: carson.u.washington.edu  A fair number of 

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**#Single Quotes Removal**

In [6]:
def remove_single_quotes(text):
    # Define the regex pattern for detecting single quotes
    pattern = r"'"
    # Replace single quotes with an empty string
    return re.sub(pattern, '', text)
df['content'] = df['content'].apply(remove_single_quotes)

# Display the cleaned texts
for index, row in df.head().iterrows():
    print(f"Text {index}: {row['content']}\n")

Text 0: From:  (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: 15   I was wondering if anyone out there could enlighten me on this car I saw the other day. It was a 2-door sports car, looked to be from the late 60s/ early 70s. It was called a Bricklin. The doors were really small. In addition, the front bumper was separate from the rest of the body. This is  all I know. If anyone can tellme a model name, engine specs, years of production, where this car is made, history, or whatever info you have on this funky looking car, please e-mail.  Thanks, - IL    ---- brought to you by your neighborhood Lerxst ----     

Text 1: From:  (Guy Kuo) Subject: SI Clock Poll - Final Call Summary: Final call for SI clock reports Keywords: SI,acceleration,clock,upgrade Article-I.D.: shelley.1qvfo9INNc3s Organization: University of Washington Lines: 11 NNTP-Posting-Host: carson.u.washington.edu  A fair number of b

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#Tokenize


> **Create sent_to_words()**
    > Use gensim.utils.simple_preprocess,
    > Use generator instead of an usual function














In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
# Tokenize each sentence into words
data_words = list(sent_to_words(df['content']))
# Display the tokenized words
for i, words in enumerate(data_words[:5]):  # Displaying first 5 documents
    print(f"Document {i+1}: {words}\n")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Document 1: ['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']

Document 2: ['from

#Stop words Removal
#Extend the stop words corpus with the from, subject, re, edu, use.

In [8]:

# Define additional stop words
additional_stopwords = {'from', 'subject', 're', 'edu', 'use'}
# Extend the default stop words list with additional stop words
STOPWORDS |= additional_stopwords
def sent_to_words(sentences):
    for sentence in sentences:
        yield [word for word in simple_preprocess(str(sentence), deacc=True) if word not in STOPWORDS]
data_words = sent_to_words(df['content'])
# Display the tokenized words
for i, words in enumerate(data_words):  # Iterate through the generator
    if i < 5:  # Displaying first 5 documents
        print(f"Document {i+1}: {words}\n")
    else:
        break  # Stop iterating after the first 5 documents

Document 1: ['wheres', 'thing', 'car', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'organization', 'university', 'maryland', 'college', 'park', 'lines', 'wondering', 'enlighten', 'car', 'saw', 'day', 'door', 'sports', 'car', 'looked', 'late', 'early', 'called', 'bricklin', 'doors', 'small', 'addition', 'bumper', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'engine', 'specs', 'years', 'production', 'car', 'history', 'info', 'funky', 'looking', 'car', 'mail', 'thanks', 'il', 'brought', 'neighborhood', 'lerxst']

Document 2: ['guy', 'kuo', 'si', 'clock', 'poll', 'final', 'summary', 'final', 'si', 'clock', 'reports', 'keywords', 'si', 'acceleration', 'clock', 'upgrade', 'article', 'shelley', 'qvfo', 'innc', 'organization', 'university', 'washington', 'lines', 'nntp', 'posting', 'host', 'carson', 'washington', 'fair', 'number', 'brave', 'souls', 'upgraded', 'si', 'clock', 'oscillator', 'shared', 'experiences', 'poll', 'send', 'brief', 'message', 'detailing', 'experiences', 'pr

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**#remove_stopwords( )**

In [9]:
def remove_stopwords(words):
    return [word for word in words if word not in STOPWORDS]
# Tokenize each sentence into words and remove stop words using the generator function
data_words = sent_to_words(df['content'])

# Display the tokenized words for the first 5 documents
for i, words in enumerate(data_words):
    if i < 5:
        print(f"Document {i+1}: {words}\n")
    else:
        break

Document 1: ['wheres', 'thing', 'car', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'organization', 'university', 'maryland', 'college', 'park', 'lines', 'wondering', 'enlighten', 'car', 'saw', 'day', 'door', 'sports', 'car', 'looked', 'late', 'early', 'called', 'bricklin', 'doors', 'small', 'addition', 'bumper', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'engine', 'specs', 'years', 'production', 'car', 'history', 'info', 'funky', 'looking', 'car', 'mail', 'thanks', 'il', 'brought', 'neighborhood', 'lerxst']

Document 2: ['guy', 'kuo', 'si', 'clock', 'poll', 'final', 'summary', 'final', 'si', 'clock', 'reports', 'keywords', 'si', 'acceleration', 'clock', 'upgrade', 'article', 'shelley', 'qvfo', 'innc', 'organization', 'university', 'washington', 'lines', 'nntp', 'posting', 'host', 'carson', 'washington', 'fair', 'number', 'brave', 'souls', 'upgraded', 'si', 'clock', 'oscillator', 'shared', 'experiences', 'poll', 'send', 'brief', 'message', 'detailing', 'experiences', 'pr

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#Bigrams

In [10]:
# Function to create bigrams
def make_bigrams(words):
    bigram_model = Phrases(words, min_count=5, threshold=100)
    data_words_bigrams = list(bigram_model[words])
    return data_words_bigrams

# Apply make_bigrams function to tokenized words
data_words_bigrams = make_bigrams(words)

# Display the first 10 documents with bigrams
for i in range(10):
    print(f"Document {i+1} with bigrams:")
    print(data_words_bigrams[i])
    print()


Document 1 with bigrams:
foxvog

Document 2 with bigrams:
douglas

Document 3 with bigrams:
rewording

Document 4 with bigrams:
second

Document 5 with bigrams:
amendment

Document 6 with bigrams:
ideas

Document 7 with bigrams:
organization

Document 8 with bigrams:
vtt

Document 9 with bigrams:
lines

Document 10 with bigrams:
article



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**#Lemmatization**

In [ ]:
! python -m spacy download en

In [24]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
# Function for lemmatization
def lemmatization(sentences, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    lemmatized_sentences = []
    for sentence in sentences:
        doc = nlp(" ".join(sentence))
        lemmatized_words = [token.lemma_ for token in doc if token.pos_ in allowed_postags and token.is_alpha]
        lemmatized_sentences.append(lemmatized_words)
    return lemmatized_sentences

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [25]:
# Tokenize each sentence into words
data_words = list(sent_to_words(df['content']))

# Define additional stop words
additional_stopwords = {'from', 'subject', 're', 'edu', 'use'}

# Extend the default stop words list with additional stop words
STOPWORDS |= additional_stopwords

# Remove stop words from tokenized words
data_words_nostops = [remove_stopwords(words) for words in data_words]

# Create bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Apply lemmatization function to tokenized words with bigrams
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(data_lemmatized[:1])  # Printing lemmatized data for the first document

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['s', 'thing', 'car', 'host', 'university', 'park', 'line', 'wonder', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'door', 'small', 'addition', 'bumper', 'separate', 'rest', 'body', 'know', 'model', 'engine', 'spec', 'year', 'production', 'car', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


**#Create a Dictionary**

In [26]:
preprocessed_data = {
    "num_documents": len(df),
    "data_words": data_words,  # Tokenized words
    "data_words_bigrams": data_words_bigrams,  # Words with bigrams
    "data_lemmatized": data_lemmatized  # Lemmatized words
}


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
# Display the dictionary keys and their corresponding lengths
for key, value in preprocessed_data.items():
    if isinstance(value, list):
        print(f"{key}: {len(value)}")
    elif isinstance(value, int):
        print(f"{key}: {value}")

num_documents: 11314
data_words: 11314
data_words_bigrams: 11314
data_lemmatized: 11314


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**#Create Corpus**

In [28]:
# Create a list to store the corpus
corpus = []
# Concatenate the preprocessed documents into the corpus
for key, value in preprocessed_data.items():
    if isinstance(value, list):
        corpus.extend(value)

# Display the length of the corpus
print("Length of the corpus:", len(corpus))

Length of the corpus: 33942


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**#Filter low-frequency words**

In [29]:
# Function to count the frequency of words in the corpus
def count_word_frequency(corpus):
    word_freq = defaultdict(int)
    for document in corpus:
        for word in document:
            word_freq[word] += 1
    return word_freq
# Function to filter out low-frequency words from the corpus based on a threshold
def filter_low_frequency_words(corpus, threshold):
    word_freq = count_word_frequency(corpus)
    filtered_corpus = [[word for word in document if word_freq[word] >= threshold] for document in corpus]
    return filtered_corpus
# Define the threshold for filtering low-frequency words
threshold = 5

# Filter out low-frequency words from the corpus
filtered_corpus = filter_low_frequency_words(corpus, threshold)

# Display the length of the filtered corpus
print("Length of the filtered corpus:", len(filtered_corpus))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Length of the filtered corpus: 33942


**#Create Index 2 word dictionary**

In [30]:
# Create a set of unique words in the corpus
unique_words = set(word for document in filtered_corpus for word in document)

# Create a dictionary to map index to word
index_to_word = {index: word for index, word in enumerate(unique_words)}

# Display the first few entries of the index-to-word dictionary
print("Index to word dictionary:")
for index in range(10):
    print(index, index_to_word[index])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Index to word dictionary:
0 nazi
1 ha
2 disassociate
3 mercilessly
4 capslock_numlock
5 deserts
6 observant
7 fractal
8 ronald_reagan
9 nuclear_plants


#Build a News Topic Model- LdaModel

In [31]:
# Create a dictionary
dictionary = corpora.Dictionary(filtered_corpus)

# Create a bag-of-words corpus
corpus = [dictionary.doc2bow(document) for document in filtered_corpus]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
# Define parameters
num_topics = 10
chunksize = 2000
passes = 20
alpha = 'auto'
# Train the LDA model
lda_model = models.LdaModel(corpus=corpus,
                            id2word=dictionary,
                            num_topics=num_topics,
                            chunksize=chunksize,
                            alpha=alpha,
                            passes=passes)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**#Print the Keyword in the 10 topics**

In [33]:
print("Keywords in the 10 topics:")
for topic_id, topic_keywords in lda_model.print_topics(num_topics=10):
    print(f"Topic {topic_id}: {topic_keywords}")

Keywords in the 10 topics:
Topic 0: 0.018*"line" + 0.017*"write" + 0.011*"article" + 0.011*"people" + 0.009*"think" + 0.009*"know" + 0.008*"say" + 0.007*"m" + 0.007*"time" + 0.006*"go"
Topic 1: 0.017*"car" + 0.013*"organization" + 0.011*"article" + 0.007*"bike" + 0.006*"host" + 0.006*"good" + 0.006*"lines" + 0.005*"know" + 0.004*"time" + 0.004*"ride"
Topic 2: 0.012*"god" + 0.009*"believe" + 0.008*"religion" + 0.006*"think" + 0.006*"people" + 0.006*"faith" + 0.006*"church" + 0.006*"word" + 0.005*"exist" + 0.005*"question"
Topic 3: 0.714*"ax" + 0.050*"max" + 0.003*"ei" + 0.002*"pl" + 0.002*"wm" + 0.002*"tm" + 0.001*"rk" + 0.001*"di" + 0.001*"mk" + 0.001*"um"
Topic 4: 0.022*"game" + 0.020*"team" + 0.013*"year" + 0.012*"play" + 0.011*"player" + 0.010*"win" + 0.009*"season" + 0.008*"hockey" + 0.007*"good" + 0.006*"organization"
Topic 5: 0.029*"key" + 0.012*"encryption" + 0.011*"information" + 0.010*"public" + 0.009*"security" + 0.008*"chip" + 0.007*"clipper" + 0.007*"government" + 0.006*"te

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#Evaluation of Topic Models--Model Perplexity

In [34]:
# Compute the perplexity of the model
perplexity = lda_model.log_perplexity(corpus)
print(f"Perplexity: {perplexity}")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Perplexity: -8.594389960955096


**#Topic Coherence**

In [35]:
# Compute the coherence score for the LDA model
coherence_model_lda = CoherenceModel(model=lda_model, texts=filtered_corpus, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()

print(f"Topic Coherence Score: {coherence_lda}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Topic Coherence Score: 0.6665321169783408


**#Visualize the Topic Model**

In [36]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Create the visualization
vis_data = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis_data)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
